In [1]:
# 서울버스정류장
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)

# 2021.09.23일 수집 완료
query = ("select seoulsttsName from public_seoulbusstts ps where 9025 <= seq and seq <= 10025")

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
now = datetime.now()
nowDate = now.strftime('%Y%m%d')

cnt = 1
for i in location_list:
    seoulsttsName = i
    
    characters = "(',')"

    join_data = ''.join( x for x in seoulsttsName if x not in characters)
    print("조회날짜 : {} | 순번 : {} | start {} public_seoulbusstts".format(nowDate, cnt, join_data))
    url = 'http://ws.bus.go.kr/api/rest/stationinfo/getStationByName'
    queryParams = '?' + urlencode({
        quote_plus('serviceKey') : public_serviceKey(),
        quote_plus('stSrch') : join_data
    })

    response = requests.get(url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

    rows = xmlobj.findAll('itemList')
    resultCode = xmlobj.findAll('headerCd')

    if resultCode[0].text == '7':
        print("{} 번째부터 조회수 제한이 걸렸습니다.".format(seoulsttsName))
        mydb.close()
        break
    cnt += 1
    for j in rows:
        item = j
        arsId = item.findAll('arsId')
        stId = item.findAll('stId')
        stNm = item.findAll('stNm')
        lat = item.findAll('tmY')
        lng = item.findAll('tmX')
        
        try:
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
        except IndexError:
            print(item)
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
            
        cursor.execute(sql, val)
        mydb.commit()

조회날짜 : 20210923 | 순번 : 1 | start 대한항공훈련센터 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 2 | start 여의나루역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 3 | start 당산역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 4 | start 은성병원 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 5 | start 동작상떼빌아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 6 | start 신대방경남아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 7 | start 신대방현대아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 8 | start 보라매공원 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 9 | start 보라매역.동작세무서 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 10 | start 강남중학교 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 11 | start 현대아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 12 | start 현대아파트.유한양행 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 13 | start 노량진수산시장.CTS기독교TV public_seoulbusstts
조회날짜 : 20210923 | 순번 : 14 | start 노량진수산시장.CTS기독교TV public_seoulbusstts
조회날짜 : 20210923 | 순번 : 15 | start 노량진역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 16 | start 노량진역 public_seoulbusstts
조회날짜 : 202

조회날짜 : 20210923 | 순번 : 132 | start 상도3차삼성래미안후문 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 133 | start 상도3차삼성래미안후문 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 134 | start 상도1동주민센터.상도3차래미안 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 135 | start 상도1동주민센터.상도3차래미안 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 136 | start 노들역앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 137 | start 대방역1번출구앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 138 | start 대방역.여의교앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 139 | start 노들역4번출구앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 140 | start 동작실버센터입구 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 141 | start 사당동대아신협앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 142 | start 사당역9번출구앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 143 | start 사당롯데캐슬골든포레.사당해그린 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 144 | start 사당솔밭도서관 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 145 | start 효주파크빌 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 146 | start 남성초등학교입구 public_seoulbusstts
조회날짜 : 20210923 | 

조회날짜 : 20210923 | 순번 : 264 | start 부강탕 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 265 | start 대방역1번출구 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 266 | start 극동아파트앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 267 | start 이수역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 268 | start 삼거리 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 269 | start 전문건설회관.보라매병원 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 270 | start 대림아파트후문 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 271 | start 상도브라운스톤 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 272 | start 노들역5번출구 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 273 | start 대림아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 274 | start 상도역.하나은행 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 275 | start 봉천고개관악현대아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 276 | start 동작삼일수영장 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 277 | start 동작고교.사당5동주민센터 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 278 | start 건영아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 279 | start 우성201동.극동112동 public_

조회날짜 : 20210923 | 순번 : 395 | start 7호선신대방삼거리역2번출구.국민은행 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 396 | start 푸른초장교회 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 397 | start 백경세탁소 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 398 | start 빙수골공영주차장 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 399 | start 선재유통 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 400 | start 영은교회 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 401 | start 갑을명가아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 402 | start 당곡중.고등학교 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 403 | start 사자암 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 404 | start 동작하이슈퍼 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 405 | start 제일성민교회 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 406 | start 보문사 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 407 | start 동아아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 408 | start 행복유치원 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 409 | start 동아아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 410 | start 보문사 public_seoulbusstts
조회날짜 : 

조회날짜 : 20210923 | 순번 : 526 | start 관악구청 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 527 | start 청룡시장 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 528 | start 영락고교.영락의료과학고 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 529 | start 쑥고개시장 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 530 | start 쑥고개 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 531 | start 은천교 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 532 | start 봉일시장 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 533 | start 은천초등학교 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 534 | start 두산.벽산아파트.서울관광고등학교 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 535 | start 현대시장앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 536 | start 관악동부센트레빌아파트앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 537 | start 중앙동성당앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 538 | start 중앙동성당앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 539 | start 관악동부센트레빌아파트앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 540 | start 벽산.두산아파트.서울관광고등학교 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 541 | start 은천초등학교 pu

조회날짜 : 20210923 | 순번 : 656 | start 청림동현대아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 657 | start 청림동현대아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 658 | start 보성운수기종점 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 659 | start 난곡공영차고지 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 660 | start 한남운수대학동차고지 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 661 | start 한남운수대학동차고지 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 662 | start 제2공학관 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 663 | start 건설환경종합연구소앞 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 664 | start 봉일시장 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 665 | start 보라매공영주차장 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 666 | start 당곡중고교 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 667 | start 연희빌라 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 668 | start 재넘이고개 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 669 | start 연희빌라 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 670 | start 당곡중고교 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 671 | start 보라매공영주차장 public_seoulbuss

조회날짜 : 20210923 | 순번 : 789 | start 관악우체국 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 790 | start 신림고등학교 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 791 | start 신림사거리.신원시장 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 792 | start 삼성동시장 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 793 | start 난곡사거리 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 794 | start 신림역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 795 | start 인헌아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 796 | start 삼성동시장입구 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 797 | start 동부아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 798 | start 낙성대역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 799 | start 당곡사거리 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 800 | start 서울과학전시관 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 801 | start 은천교입구 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 802 | start 은천교입구 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 803 | start 삼성아파트 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 804 | start 조원경로당 public_seoulbusstts
조회날짜 : 20210923 |

조회날짜 : 20210923 | 순번 : 920 | start 논현역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 921 | start 신사역.푸른저축은행 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 922 | start 신사역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 923 | start 반포역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 924 | start 반포역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 925 | start 고속터미널 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 926 | start 고속터미널 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 927 | start 신반포역.세화여중고 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 928 | start 신반포역.세화여중고 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 929 | start 구반포역.세화고등학교 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 930 | start 구반포역.세화고등학교 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 931 | start 아리랑TV.국립국악원 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 932 | start 남태령역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 933 | start 이수역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 934 | start 사당역 public_seoulbusstts
조회날짜 : 20210923 | 순번 : 935 | start 총신대입구역.남성시장입구앞 public_seoulbus